In [1]:
import pandas as pd
import os
from find_vehicle_image_urls import *

In [2]:
def compile_search_results_df() ->pd.DataFrame:
    '''
    :return:
    DF where each row represents on vehicle listing with a unique url and vehicle id

    Also contains make/model of the SEARCH from which the url was captured
    NB: This might not be the make/model of the actual vehicle;
     The vehicle listing page is the ultimate source of truth on the make/model of the vehicle

    '''
    ## compile search results from multiple csvs intoo one df
    # folder="/Users/levgolod/Projects/car_classifier/data/autotrader/vehicle_metadata/"
    folder=parent_directory_url_csvs
    pattern=r'search_results_.*[0-9]*csv'
    usecols=['listing_header',
     'url',
     'search_url',
     'make',
     'model',
     'search_timestamp',
     'search_metadata']
    # bigdf= pd.DataFrame(columns=usecols)
    bigdf= None

    files = [x for x in os.listdir(folder) if bool(re.search(pattern, x))]

    # file=files[0]
    for file in files:
        print(file)
        try:
            df=pd.read_csv(folder+file, usecols=usecols)
            df=df[usecols]
            df['filename']=file
        except ValueError:
            print(f'File {file} wrong columns, skipping')
            continue


        bigdf = pd.concat([bigdf, df], ignore_index=True) if bigdf is not None else df
        # bigdf=bigdf.drop_duplicates(subset=['url'])
        print(bigdf.shape)

    bigdf['url_clean'] = bigdf['url'].apply(clean_vehicle_url)
    bigdf['vehicle_id'] = bigdf['url_clean'].apply(lambda x: str(x).split('/')[-1] )

    bigdf.head()
    return bigdf

search_results_df=compile_search_results_df()
search_results_df.head()


search_results_honda_odyssey_1739360198.csv
(31, 8)
search_results_alfa romeo_4c.csv.csv
File search_results_alfa romeo_4c.csv.csv wrong columns, skipping
search_results_ford_explorer_1739359570.csv
(61, 8)
search_results_ford_taurus_1739325006.csv
(90, 8)
search_results_porsche_taycan_1739322969.csv
File search_results_porsche_taycan_1739322969.csv wrong columns, skipping
search_results_bmw_x5_1739338391.csv
(119, 8)
search_results_bmw_4-series_1739371650.csv
(148, 8)
search_results_toyota_tacoma_1739356759.csv
(178, 8)
search_results_toyota_corolla_1739352090.csv
(208, 8)
search_results_toyota_crown_1739367061.csv
(238, 8)
search_results_honda_ridgeline_1739327606.csv
(269, 8)
search_results_honda_civic_1739350203.csv
(300, 8)
search_results_bmw_3-series_1739352708.csv
(329, 8)
search_results_bmw_x5_1739363630.csv
(358, 8)
search_results_toyota_crown_1739356445.csv
(388, 8)
search_results_porsche_taycan_1739323122.csv
File search_results_porsche_taycan_1739323122.csv wrong columns, s

,listing_header,url,search_url,make,model,search_timestamp,search_metadata,filename,url_clean,vehicle_id
0,Used 2020 Honda Odyssey EX-L,https://www.autotrader.com/cars-for-sale/vehic...,https://www.autotrader.com/cars-for-sale/honda...,honda,odyssey,1739360198,"{'make': 'honda', 'model': 'odyssey'}",search_results_honda_odyssey_1739360198.csv,https://www.autotrader.com/cars-for-sale/vehic...,732504624
1,Used 2011 Honda Odyssey EX-L,https://www.autotrader.com/cars-for-sale/vehic...,https://www.autotrader.com/cars-for-sale/honda...,honda,odyssey,1739360198,"{'make': 'honda', 'model': 'odyssey'}",search_results_honda_odyssey_1739360198.csv,https://www.autotrader.com/cars-for-sale/vehic...,738409771
2,Used 2024 Honda Odyssey Sport,https://www.autotrader.com/cars-for-sale/vehic...,https://www.autotrader.com/cars-for-sale/honda...,honda,odyssey,1739360198,"{'make': 'honda', 'model': 'odyssey'}",search_results_honda_odyssey_1739360198.csv,https://www.autotrader.com/cars-for-sale/vehic...,736531071
3,Certified 2024 Honda Odyssey EX-L,https://www.autotrader.com/cars-for-sale/vehic...,https://www.autotrader.com/cars-for-sale/honda...,honda,odyssey,1739360198,"{'make': 'honda', 'model': 'odyssey'}",search_results_honda_odyssey_1739360198.csv,https://www.autotrader.com/cars-for-sale/vehic...,731560875
4,Used 2019 Honda Odyssey EX-L,https://www.autotrader.com/cars-for-sale/vehic...,https://www.autotrader.com/cars-for-sale/honda...,honda,odyssey,1739360198,"{'make': 'honda', 'model': 'odyssey'}",search_results_honda_odyssey_1739360198.csv,https://www.autotrader.com/cars-for-sale/vehic...,736627883


In [3]:
## let's learn how many vehicle urls are provided by the typical search
print(search_results_df.groupby(['filename']).size().describe())
np.mean(search_results_df.groupby(['filename']).size() >=20)

count    116.000000
mean      29.913793
std        0.786505
min       29.000000
25%       29.000000
50%       30.000000
75%       31.000000
max       31.000000
dtype: float64


1.0

In [4]:
## dedup (devastating)
n_before = len(search_results_df)
search_results_df=search_results_df.drop_duplicates(subset=['url'])
n_after = len(search_results_df)
message =f'after deduplicating I go from {n_before} to {n_after} records'
print(message)


## let's learn how many vehicle urls are provided by the typical search

print(search_results_df.groupby(['filename']).size().describe())
np.mean(search_results_df.groupby(['filename']).size() >=20)

after deduplicating I go from 3470 to 973 records
count    116.000000
mean       8.387931
std        9.319238
min        2.000000
25%        4.000000
50%        5.000000
75%        5.000000
max       31.000000
dtype: float64


0.15517241379310345

In [5]:

# df, _ = process_vehicle_webpage(search_results_df['url'][1])

In [6]:
i=2

search_results_df.loc[i, ]
vehicle_id = search_results_df.loc[i, 'vehicle_id']
filepath=f'{parent_directory_url_csvs}{vehicle_id}.csv'
filepath
already_exist = os.path.exists(filepath)
filepath, already_exist


if already_exist:
    print(f'file {filepath} already exists, skipping')
else:
    df, _ = process_vehicle_webpage(search_results_df.loc[i, 'url_clean'])


file ./data/autotrader/vehicle_metadata/736531071.csv already exists, skipping


In [7]:
!ls -lsh ~/Projects/car_classifier/data/autotrader/vehicle_metadata/738409771.csv

24 -rw-r--r--@ 1 levgolod  staff   9.9K Feb 12 09:09 /Users/levgolod/Projects/car_classifier/data/autotrader/vehicle_metadata/738409771.csv


In [8]:
search_results_df.sample(frac=1)
search_results_df.reset_index(drop=True)
sleep=60

for i in search_results_df.index:

    search_results_df.loc[i, ]
    vehicle_id = search_results_df.loc[i, 'vehicle_id']
    url_clean = search_results_df.loc[i, 'url_clean']
    filepath=f'{parent_directory_url_csvs}{vehicle_id}.csv'
    filepath
    already_exist = os.path.exists(filepath)
    filepath, already_exist


    if already_exist:
        print(f'file {filepath} already exists, skipping')
        continue

    try:
        with timethis(url_clean):
            df, _ = process_vehicle_webpage(url_clean)

        print(f'sleeping {sleep} seconds')
        time.sleep(sleep)
    except Exception as e:
        error_message = traceback.format_exc()
        print(f"[{url_clean}] Error Traceback:\n", error_message)


file ./data/autotrader/vehicle_metadata/732504624.csv already exists, skipping
file ./data/autotrader/vehicle_metadata/738409771.csv already exists, skipping
file ./data/autotrader/vehicle_metadata/736531071.csv already exists, skipping
file ./data/autotrader/vehicle_metadata/731560875.csv already exists, skipping
file ./data/autotrader/vehicle_metadata/736627883.csv already exists, skipping
file ./data/autotrader/vehicle_metadata/737511994.csv already exists, skipping
file ./data/autotrader/vehicle_metadata/737175430.csv already exists, skipping
single vehicle listing [https://www.autotrader.com/cars-for-sale/vehicle/733365286]- webpage initiated
https://www.autotrader.com/cars-for-sale/vehicle/733365286: 19.608 seconds
[https://www.autotrader.com/cars-for-sale/vehicle/733365286] Error Traceback:
 Traceback (most recent call last):
  File "/var/folders/t9/qz0dbkyj1hn5b4g44hx59v8r0000gq/T/ipykernel_32511/2453251788.py", line 22, in <module>
    df, _ = process_vehicle_webpage(url_clean

KeyboardInterrupt: 

In [ ]:
# single vehicle listing [https://www.autotrader.com/cars-for-sale/vehicle/725617155]- webpage initiated
# doesn't have the View All Media button bc there are only 2 photos



In [ ]:
'''
We're sorry for any inconvenience, but the site is currently unavailable.
'''